<h1><center> Projet Machine Learning avec Python : </center></h1>
<h1><center> Prédiction des incidents  à San Francisco </center></h1>

 

<h1><center> Partie 3 : Prédiction multiple du nombre d'incidents par catégorie et par heure <center></h1>

In [70]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(11, 4)})

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import MaxAbsScaler, StandardScaler, RobustScaler, MinMaxScaler
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error,  make_scorer
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC

from sklearn.metrics import f1_score

from xgboost import XGBRegressor
from xgboost import XGBClassifier

# 1. Opérations préliminaires sur la base de données

In [2]:
df = pd.read_csv('Fire_Department_Calls_for_Service_reduced_V3.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


**1.1. Nombre d'appels par heure, date et lieu**

In [3]:
count_test = df.groupby(['year', 'month', 'day', 'hour', 'weekday', 'Neighborhooods - Analysis Boundaries']).size().reset_index(name='count')

In [4]:
df.head(5)

,Unnamed: 0,Call Date,Received DtTm,Address,Zipcode of Incident,Battalion,Station Area,Box,Final Priority,Neighborhooods - Analysis Boundaries,Location,shape,Neighborhoods - Analysis Boundaries,year,month,day,hour,minute,weekday,Call Category
0,2949367,2012-01-01 00:00:00,2012-01-01 00:00:57,0 Block of WASHINGTON ST,94111.0,B01,13.0,1116.0,2,Financial District/South Beach,"(37.796450111202, -122.396164646182)",POINT (-122.396164646182 37.796450111202),8.0,2012,1,1,0,0,Sunday,Medical Non Life-Threatening
1,2920610,2012-01-01 00:00:00,2012-01-01 00:02:05,GEARY BL/CLEARY CT,94109.0,B04,3.0,3323.0,2,Western Addition,"(37.785061737122, -122.42643012773)",POINT (-122.42643012773 37.785061737122),39.0,2012,1,1,0,2,Sunday,Medical Non Life-Threatening
2,3003150,2012-01-01 00:00:00,2012-01-01 00:02:27,HAIGHT ST/LAGUNA ST,94102.0,B02,36.0,3415.0,3,Hayes Valley,"(37.772734311086, -122.425512056036)",POINT (-122.425512056036 37.772734311086),9.0,2012,1,1,0,2,Sunday,Fire
3,2990241,2012-01-01 00:00:00,2012-01-01 00:02:53,WALLER ST/LAGUNA ST,94102.0,B02,36.0,3415.0,3,Hayes Valley,"(37.771795346154, -122.425322486842)",POINT (-122.425322486842 37.771795346154),9.0,2012,1,1,0,2,Sunday,Fire
4,3002275,2012-01-01 00:00:00,2012-01-01 00:03:55,1100 Block of FOLSOM ST,94103.0,B02,29.0,2322.0,3,South of Market,"(37.775260329348, -122.409820946894)",POINT (-122.409820946894 37.775260329348),34.0,2012,1,1,0,3,Sunday,Medical Potentially Life-Threatening


In [5]:
count_test.head(5)

,year,month,day,hour,weekday,Neighborhooods - Analysis Boundaries,count
0,2012,1,1,0,Sunday,Bayview Hunters Point,2
1,2012,1,1,0,Sunday,Bernal Heights,1
2,2012,1,1,0,Sunday,Castro/Upper Market,1
3,2012,1,1,0,Sunday,Financial District/South Beach,23
4,2012,1,1,0,Sunday,Haight Ashbury,1


**1.2. Nombre d'appels par catégorie par heure, date et lieu**

In [6]:
dum = pd.get_dummies(df[['year', 'month', 'day', 'hour', 'weekday', 'Neighborhooods - Analysis Boundaries', 
                         'Call Category']], columns=['Call Category'])

dum2 = dum.groupby(['year', 'month', 'day', 'hour', 'weekday',
                    'Neighborhooods - Analysis Boundaries']).sum().reset_index()

In [7]:
dum2

,year,month,day,hour,weekday,Neighborhooods - Analysis Boundaries,Call Category_Alarms,Call Category_Fire,Call Category_Incidents,Call Category_Medical Non Life-Threatening,Call Category_Medical Potentially Life-Threatening,Call Category_Other
0,2012,1,1,0,Sunday,Bayview Hunters Point,0,1,0,1,0,0
1,2012,1,1,0,Sunday,Bernal Heights,0,0,0,0,1,0
2,2012,1,1,0,Sunday,Castro/Upper Market,0,0,1,0,0,0
3,2012,1,1,0,Sunday,Financial District/South Beach,0,1,2,6,13,1
4,2012,1,1,0,Sunday,Haight Ashbury,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
793071,2020,9,2,23,Wednesday,Pacific Heights,0,0,0,1,0,0
793072,2020,9,2,23,Wednesday,Russian Hill,0,0,0,0,1,0
793073,2020,9,2,23,Wednesday,South of Market,0,0,0,1,0,0
793074,2020,9,2,23,Wednesday,Tenderloin,0,0,1,0,0,0


Dans notre base nous avons des heures manquantes (celles durant lesquelles aucun appel n'a été émis). Dans ce qui suit, afin que toutes les heures soient représentées, nous transformons notre base en ajoutant les heures inexistentes en affectant zéro comme nombre d'appels durant ces heures.

**1.3. Création d'un dataframe contenant toutes les heures entre le 01/01/2012 et le 31/12/2020**

In [8]:
df_2012_2020 = pd.DataFrame(
        {'Received DtTm': pd.date_range('2012-01-01', '2021-01-01', freq='1H', closed='left')})

In [9]:
df_2012_2020

,Received DtTm
0,2012-01-01 00:00:00
1,2012-01-01 01:00:00
2,2012-01-01 02:00:00
3,2012-01-01 03:00:00
4,2012-01-01 04:00:00
...,...
78907,2020-12-31 19:00:00
78908,2020-12-31 20:00:00
78909,2020-12-31 21:00:00
78910,2020-12-31 22:00:00


**1.4. Produit cartésien entre les heures/dates et les 41 quartiers de San Francisco**

In [10]:
# Pour chaque date il affiche les 41 quartiers
df_cartesian = df_2012_2020.merge(pd.DataFrame(df['Neighborhooods - Analysis Boundaries'].unique(),
                                               columns=["Neighborhooods - Analysis Boundaries"]), how='cross')
df_cartesian.head(10)

,Received DtTm,Neighborhooods - Analysis Boundaries
0,2012-01-01,Financial District/South Beach
1,2012-01-01,Western Addition
2,2012-01-01,Hayes Valley
3,2012-01-01,South of Market
4,2012-01-01,Treasure Island
5,2012-01-01,Bernal Heights
6,2012-01-01,Mission
7,2012-01-01,Marina
8,2012-01-01,Inner Sunset
9,2012-01-01,North Beach


In [11]:
df_cartesian = df_2012_2020.merge(pd.DataFrame(df['Neighborhooods - Analysis Boundaries'].unique(),
                                               columns=["Neighborhooods - Analysis Boundaries"]), how='cross')

df_cartesian['year'] = df_cartesian['Received DtTm'].dt.year
df_cartesian['month'] = df_cartesian['Received DtTm'].dt.month
df_cartesian['day'] = df_cartesian['Received DtTm'].dt.day
df_cartesian['hour'] = df_cartesian['Received DtTm'].dt.hour
df_cartesian["weekday"] = df_cartesian['Received DtTm'].dt.day_name()


df_cartesian = df_cartesian.drop(columns='Received DtTm')

In [12]:
df_cartesian.shape

(3235392, 6)

In [13]:
df_2012_2020.shape[0]*41

3235392

**1.5. Fusion de nos 2 bases de données afin d'avoir pour chaque heure, date et lieu, le nombre d'appels (qui peut donc être nul)**

Nous fusionnons les deux bases afin d'avoir toutes les dates et toutes les heures et le nombre d'incidents 
par heure : si la ligne (heure, jour, année,...) existe dans count_test on aura le nombre d'appel, si la ligne
n'existe pas on aura une variable manquante.

In [14]:
result = pd.merge(df_cartesian, count_test, on=["year", "month", "day", "hour",
                                                "weekday", "Neighborhooods - Analysis Boundaries"], how='left')

result = result.fillna(0) # Remplacer les valeurs manquantes par zéro

result

,Neighborhooods - Analysis Boundaries,year,month,day,hour,weekday,count
0,Financial District/South Beach,2012,1,1,0,Sunday,23.0
1,Western Addition,2012,1,1,0,Sunday,3.0
2,Hayes Valley,2012,1,1,0,Sunday,4.0
3,South of Market,2012,1,1,0,Sunday,4.0
4,Treasure Island,2012,1,1,0,Sunday,2.0
...,...,...,...,...,...,...,...
3235387,Presidio Heights,2020,12,31,23,Thursday,0.0
3235388,Seacliff,2020,12,31,23,Thursday,0.0
3235389,Glen Park,2020,12,31,23,Thursday,0.0
3235390,McLaren Park,2020,12,31,23,Thursday,0.0


**1.6. Fusion entre les 2 bases de données dans le même but que précédemment**

On aura toutes les heures des 8 années avec le nombre d'appels par heure.

In [15]:
result_dum = pd.merge(df_cartesian, dum2, on=["year", "month", "day", "hour", "weekday",
                                              "Neighborhooods - Analysis Boundaries"], how='left')

result_dum = result_dum.fillna(0)

result_dum

,Neighborhooods - Analysis Boundaries,year,month,day,hour,weekday,Call Category_Alarms,Call Category_Fire,Call Category_Incidents,Call Category_Medical Non Life-Threatening,Call Category_Medical Potentially Life-Threatening,Call Category_Other
0,Financial District/South Beach,2012,1,1,0,Sunday,0.0,1.0,2.0,6.0,13.0,1.0
1,Western Addition,2012,1,1,0,Sunday,0.0,0.0,0.0,2.0,1.0,0.0
2,Hayes Valley,2012,1,1,0,Sunday,0.0,4.0,0.0,0.0,0.0,0.0
3,South of Market,2012,1,1,0,Sunday,0.0,0.0,0.0,2.0,2.0,0.0
4,Treasure Island,2012,1,1,0,Sunday,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
3235387,Presidio Heights,2020,12,31,23,Thursday,0.0,0.0,0.0,0.0,0.0,0.0
3235388,Seacliff,2020,12,31,23,Thursday,0.0,0.0,0.0,0.0,0.0,0.0
3235389,Glen Park,2020,12,31,23,Thursday,0.0,0.0,0.0,0.0,0.0,0.0
3235390,McLaren Park,2020,12,31,23,Thursday,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
df2=result_dum.copy()

**1.7. Notre base de données est très grande pour accélérer les algo de ML nous limitons nos données à l'année 2019**

In [17]:
df2 = df2[(df2['year']>2018) & (df2['year']<2020)]
df2.shape

(359160, 12)

In [19]:
df3=df2

# 2. Preprocessing des données

**2.1. Encodage variables catégorielles**

In [20]:
categorical_features= ['Neighborhooods - Analysis Boundaries', "weekday"]

df3[categorical_features]=df3[categorical_features].apply(LabelEncoder().fit_transform)
df3.head()

,Neighborhooods - Analysis Boundaries,year,month,day,hour,weekday,Call Category_Alarms,Call Category_Fire,Call Category_Incidents,Call Category_Medical Non Life-Threatening,Call Category_Medical Potentially Life-Threatening,Call Category_Other
2516088,5,2019,1,1,0,5,0.0,0.0,0.0,1.0,3.0,0.0
2516089,40,2019,1,1,0,5,0.0,0.0,0.0,0.0,0.0,0.0
2516090,9,2019,1,1,0,5,0.0,0.0,0.0,0.0,0.0,0.0
2516091,33,2019,1,1,0,5,0.0,0.0,0.0,0.0,1.0,0.0
2516092,36,2019,1,1,0,5,0.0,0.0,0.0,0.0,1.0,0.0


**2.2. Choix de nos variables target**

In [21]:
y_columns=['Call Category_Alarms', 'Call Category_Fire', 'Call Category_Incidents','Call Category_Medical Non Life-Threatening','Call Category_Medical Potentially Life-Threatening',
       'Call Category_Other']

In [22]:
df3["Call Category_Medical Non Life-Threatening"].dtype

dtype('float64')

In [23]:
df3[y_columns] = df3[y_columns].astype(int)


**2.3. Choix des variables numériques et catégorielles pour la prédiction**

In [97]:
categorical_features= ['Neighborhooods - Analysis Boundaries', "weekday"]
numeric_features= ["month", "day", "hour"]

#df3= df2.sample(n=50000)  #Selection aléatoire de n lignes
features = numeric_features + categorical_features
X = df3.drop(y_columns, axis=1)[features].to_numpy()
y = df3[y_columns].to_numpy()

In [98]:
X.shape

(359160, 5)

**2.4. Diviser notre jeu de données en train et test**

In [25]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)

# 3. Prédiction multiple du nombre d'incidents par catégorie et par heure

**3.1. KNN**

In [91]:
multi_target_knn = MultiOutputRegressor( KNeighborsRegressor(), n_jobs=-1)

parameters = {'estimator__n_neighbors':[1,2,3]} # defining parameter space

clf1 = GridSearchCV(multi_target_knn, param_grid=parameters, cv=3)
clf1.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf1.best_params_))

Returned hyperparameter: {'estimator__n_neighbors': 3}


**clf1 : Comparaison des valeurs prédites et des valeurs dans test**

In [92]:
y_predict=clf1.predict(X_test)
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_predict: ",sum(y_predict[:,:]>1))
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_test: ",sum(y_test[:,:]>1))

nbre de valeurs supérieures à 1 dans chaque colonne de y_predict:  [   5    4    5  583 1239    3]
nbre de valeurs supérieures à 1 dans chaque colonne de y_test:  [ 113   50   87 1405 2087   31]


**clf1 : Accuracy par composante sur le nombre d'incidents pour chaque catégorie d'appel**

L'accuracy par composante représente la proportion de bonnes prédictions sur les données de test pour chaque catégorie d'appel.

In [93]:
def Performance(Classe):
    perf=[]
    y_predict=Classe.predict(X_test)
    for i in range (y_test.shape[1]):
        perf.append(sum(y_predict[:,i]==y_test[:,i])/y_test.shape[0])
    return perf

In [94]:
Performance(clf1)

[0.8808191335338011,
 0.9229034413631808,
 0.914105134202027,
 0.6468147900657089,
 0.6003452500278428,
 0.9278872925715559]

Interprétation de la sortie précédente : le taux de bonne prédiction pour première catégorie d'appel ('Call Category_Alarms') est égale à 88%. Celui de la catégorie 'Call Category_Fire' est de 92%, celui de 'Call Category_Incidents' est de 91%, celui de 'Call Category_Medical Non Life-Threatening' est de 64%, celui de 'Call Category_Medical Potentially Life-Threatening' est de 60% et celui de'Call Category_Other' est de 92%.

**clf 1 : Accuracy sur la prédiction multiple**

L'accuracy multiple représente la proportion de bonnes prédictions (vectorielles) sur les données de test.

In [95]:
def Performance_multiple(Classe):
    y_predict=Classe.predict(X_test)
    A=y_predict==y_test
    nbre=0
    for i in range(A.shape[0]):
        if sum(A[i])==6:
            nbre+=1
    score=nbre/A.shape[0]        
    return score
print("La performance sur le prédiction vectorielle de X_test est:", Performance_multiple(clf1))

La performance sur le prédiction vectorielle de X_test est: 0.3480343022608308


Interprétation : 34% des vecteurs, composés par les nombres d'appels des six catégories d'appel par heure, ont été correctement prédits.

**3.2. KNN avec la fonction de perte mean squared error**

In [31]:
start_time = time.time()

multi_target_knn = MultiOutputRegressor( KNeighborsRegressor(), n_jobs=-1)

mean_squared_score = make_scorer(mean_squared_error) # définir la fonction de perte mean squared error


parameters = {'estimator__n_neighbors':[1,2,3]} # defining parameter space

clf2 = GridSearchCV(multi_target_knn, param_grid=parameters, cv=3, scoring=mean_squared_score)
clf2.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf2.best_params_))
print('Best mean squared error in train is: {}'.format(clf2.best_score_))
print('Mean squared error on test is: {}'.format(clf2.score(X_test, y_test)))
print("Temps d'exécution %s seconds" % (time.time() - start_time))

Returned hyperparameter: {'estimator__n_neighbors': 1}
Best mean squared error in train is: 0.1563532501763374
Mean squared error on test is: 0.15356201507220552
Temps d'exécution 32.0481538772583 seconds


**clf2 : Comparaison des valeurs prédites et des valeurs dans test**

In [99]:
y_predict=clf2.predict(X_test)
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_predict: ",sum(y_predict[:,:]>1))
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_test: ",sum(y_test[:,:]>1))

nbre de valeurs supérieures à 1 dans chaque colonne de y_predict:  [  72   63   60 1221 1829   21]
nbre de valeurs supérieures à 1 dans chaque colonne de y_test:  [ 113   50   87 1405 2087   31]


**clf2 : Performance sur la prédiction du nombre de chaque type d'appels**

In [62]:
Performance(clf2)

[0.9396647733600624,
 0.961312506960686,
 0.9559667000779597,
 0.8079964361287448,
 0.7767429557857223,
 0.9638461966811449]

Le modèle KNN avec la fonction de perte mean squared error prédit relativement mieux que le KNN avec la fonction de perte par défaut.  

**clf2 : Performance sur la prédiction vectorielle des appels**

In [63]:
print("La performance sur le prédiction vectorielle de X_test est:", Performance_multiple(clf2))

La performance sur le prédiction vectorielle de X_test est: 0.5758575565207706


**clf2 : Matrice de confusin pour la catégorie 'Call Category_Medical Non Life-Threatening'**

In [100]:
confusion_matrix(y_test[:,3], y_predict[:,3])

array([[57026,  5504,   674,   125,    24,     3,     0,     0],
       [ 5827,   959,   221,    46,    15,     2,     1,     0],
       [  833,   198,    48,    21,     4,     1,     0,     0],
       [  162,    57,    23,     7,     1,     0,     0,     0],
       [   25,    10,     1,     1,     0,     1,     0,     0],
       [    5,     2,     1,     1,     0,     0,     0,     0],
       [    1,     1,     0,     0,     0,     0,     0,     0],
       [    1,     0,     0,     0,     0,     0,     0,     0]])

D'après la matrice de confucion pour la catégorie 'Call Category_Medical Non Life-Threatening', la proportion des bonnes prédiction lorsque le nombre d'appel est nul est très important (proche de 1). Cependant, la proportion des bonnes prédiction sont très faibles pour les autres valeurs du nombre d'appel.

In [101]:
df3['Call Category_Medical Non Life-Threatening'].unique()

array([1, 0, 2, 4, 3, 5, 7, 6])

In [68]:
y_predict.shape

(71832, 6)

In [67]:
sum(y_predict==0)

array([69691, 70423, 70232, 63880, 62334, 70494])

In [66]:
sum(y_predict==[0, 0, 0, 0, 0, 0])

array([69691, 70423, 70232, 63880, 62334, 70494])

In [102]:
y_predict[:10]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

**3.3. Random Forest**

In [35]:
start_time = time.time()
multi_target_rf = MultiOutputClassifier( RandomForestClassifier(), n_jobs=-1)

parameters = {'estimator__n_estimators': [15, 50, 100]} # defining parameter space


clf3 = GridSearchCV(estimator=multi_target_rf, param_grid=parameters, cv=3)
clf3.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf3.best_params_))
print('Accuracy in train is: {}'.format(clf3.best_score_))
print('Accuracy on test is: {}'.format(clf3.score(X_test, y_test)))
print("Temps d'exécution %s seconds" % (time.time() - start_time))

Returned hyperparameter: {'estimator__n_estimators': 100}
Accuracy in train is: 0.7147580465530683
Accuracy on test is: 0.7119250473326652
Temps d'exécution 335.6296899318695 seconds


In [73]:
y_predict=clf3.predict(X_test)
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_predict: ",sum(y_predict[:,:]>1))
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_test: ",sum(y_test[:,:]>1))

nbre de valeurs supérieures à 1 dans chaque colonne de y_predict:  [  3   0   1 466 943   2]
nbre de valeurs supérieures à 1 dans chaque colonne de y_test:  [ 113   50   87 1405 2087   31]


**clf3 : Performance**

In [74]:
Performance(clf3)

[0.964848535471656,
 0.9785471656086424,
 0.9756515202138323,
 0.8728421873259828,
 0.8494821249582358,
 0.980593607305936]

**clf3 : Accuracy**

In [38]:
print("La performance sur le prédiction vectorielle de X_test est:", Performance_multiple(clf3))

La performance sur le prédiction vectorielle de X_test est: 0.7119250473326652


In [81]:
y_predict[:10]

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

**clf3 : Matrice de confusion pour la catégorie'Call Category_Medical Non Life-Threatening'**

In [90]:
confusion_matrix(y_test[:,3], y_predict[:,3])

array([[62013,  1165,   143,    29,     4,     2,     0,     0],
       [ 6287,   615,   131,    29,     8,     0,     1,     0],
       [  771,   255,    66,    10,     3,     0,     0,     0],
       [  142,    72,    32,     4,     0,     0,     0,     0],
       [   16,    20,     1,     0,     0,     1,     0,     0],
       [    3,     4,     2,     0,     0,     0,     0,     0],
       [    1,     1,     0,     0,     0,     0,     0,     0],
       [    0,     1,     0,     0,     0,     0,     0,     0]])

In [89]:
df3['Call Category_Medical Non Life-Threatening'].unique()

array([1, 0, 2, 4, 3, 5, 7, 6])

**3.4. Random Forest avec la fonction de perte mean squared error**

In [39]:
start_time = time.time()
multi_target_rf = MultiOutputClassifier( RandomForestClassifier(), n_jobs=-1)

parameters = {'estimator__n_estimators': [15, 50, 100]} # defining parameter space
mean_squared_score = make_scorer(mean_squared_error)


clf4 = GridSearchCV(estimator=multi_target_rf, param_grid=parameters, cv=3, scoring=mean_squared_score)
clf4.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf4.best_params_))
print('Best mean squared error in train is: {}'.format(clf4.best_score_))
print('Mean squared error on test is: {}'.format(clf4.score(X_test, y_test)))
print("Temps d'exécution %s seconds" % (time.time() - start_time))

Returned hyperparameter: {'estimator__n_estimators': 15}
Best mean squared error in train is: 0.0983115695882986
Mean squared error on test is: 0.09898327579166201
Temps d'exécution 265.0703778266907 seconds


In [40]:
y_predict=clf4.predict(X_test)
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_predict: ",sum(y_predict[:,:]>1))
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_test: ",sum(y_test[:,:]>1))

nbre de valeurs supérieures à 1 dans chaque colonne de y_predict:  [ 15   5   3 452 980   2]
nbre de valeurs supérieures à 1 dans chaque colonne de y_test:  [ 113   50   87 1405 2087   31]


**clf4 : Performance**

In [41]:
Performance(clf4)

[0.963010914355719,
 0.9772524780042321,
 0.9738974273304377,
 0.8676077514199799,
 0.8438439692616104,
 0.979786167724691]

**clf4 : Accuracy**

In [42]:
print("La performance sur le prédiction vectorielle de X_test est:", Performance_multiple(clf4))

La performance sur le prédiction vectorielle de X_test est: 0.7013726472881168


**3.5. Régression logistique**

In [49]:
multi_target_LR = MultiOutputClassifier( LogisticRegression(max_iter=1000), n_jobs=-1)

parameters = {'estimator__C': list(np.logspace(-2, 2, 5, base=2))} # defining parameter space

clf5 = GridSearchCV(estimator=multi_target_LR, param_grid=parameters, cv=3)
clf5.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf5.best_params_))
print('Best accuracy in train is: {}'.format(clf5.best_score_))
print('Accuracy on test is: {}'.format(clf5.score(X_test, y_test)))

Returned hyperparameter: {'estimator__C': 0.25}
Best accuracy in train is: 0.7268591992426773
Accuracy on test is: 0.7242872257489699


In [50]:
y_predict=clf5.predict(X_test)
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_predict: ",sum(y_predict[:,:]>1))
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_test: ",sum(y_test[:,:]>1))

nbre de valeurs supérieures à 1 dans chaque colonne de y_predict:  [0 0 0 0 0 0]
nbre de valeurs supérieures à 1 dans chaque colonne de y_test:  [ 113   50   87 1405 2087   31]


**clf5 : Performance**

In [51]:
Performance(clf5)

[0.966226751308609,
 0.9790622563759884,
 0.9762362178416305,
 0.8820024501614879,
 0.8580437687938524,
 0.9810530125849204]

**clf5 : Accuracy**

In [52]:
print("La performance sur le prédiction vectorielle de X_test est:", Performance_multiple(clf5))

La performance sur le prédiction vectorielle de X_test est: 0.7242872257489699


**3.6. Régression logistique avec mean squarred error**

In [53]:
multi_target_LR = MultiOutputClassifier( LogisticRegression(max_iter=1000), n_jobs=-1)
mean_squared_score = make_scorer(mean_squared_error)

parameters = {'estimator__C': list(np.logspace(-2, 2, 5, base=2))} # defining parameter space

clf6 = GridSearchCV(estimator=multi_target_LR, param_grid=parameters, cv=3, scoring=mean_squared_score)
clf6.fit(X_train, y_train)

print('Returned hyperparameter: {}'.format(clf8.best_params_))
print('Mean squared error in train is: {}'.format(clf8.best_score_))
print('Mean squared error on test is: {}'.format(clf8.score(X_test, y_test)))

Returned hyperparameter: {'estimator__C': 0.25}
Mean squared error in train is: 0.09941367821212459
Mean squared error on test is: 0.10078377324869138


In [54]:
y_predict=clf6.predict(X_test)
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_predict: ",sum(y_predict[:,:]>1))
print("nbre de valeurs supérieures à 1 dans chaque colonne de y_test: ",sum(y_test[:,:]>1))

nbre de valeurs supérieures à 1 dans chaque colonne de y_predict:  [0 0 0 0 0 0]
nbre de valeurs supérieures à 1 dans chaque colonne de y_test:  [ 113   50   87 1405 2087   31]


**clf6 : Performance**

In [55]:
Performance(clf6)

[0.966226751308609,
 0.9790622563759884,
 0.9762362178416305,
 0.8820024501614879,
 0.8580437687938524,
 0.9810530125849204]

**clf6 : Accuracy**

In [56]:
print("La performance sur le prédiction vectorielle de X_test est:", Performance_multiple(clf8))

La performance sur le prédiction vectorielle de X_test est: 0.7242872257489699


# Fin